In [1]:
# Kill all processess on GPU
# !fuser -v /dev/nvidia* -k

# Libraries

In [2]:
%%capture
import os
if 'COLAB_' not in ''.join(os.environ.keys()):
    %pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    %pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    %pip install --no-deps cut_cross_entropy unsloth_zoo
    %pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    %pip install --no-deps unsloth

In [11]:
import os
import torch  # type: ignore
# from datetime import datetime
# from datasets import load_dataset
from unsloth import FastLanguageModel # type: ignore
from transformers import TrainingArguments  # type: ignore
from trl import SFTTrainer  # type: ignore
from peft import LoraConfig # type: ignore
from huggingface_hub import snapshot_download # type: ignore
from safetensors.torch import load_file # type: ignore
from pprint import pprint

# Config

In [5]:
# Project config
seed = 69 # Nice.

# Model config
max_seq_length = 1024
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# LoRA config
hf_lora_id = 'alxxtexxr/L3.1-8B-wikipedia-en-LoRA-v20250305134947'
lora_dir = hf_lora_id.split('/')[-1]

In [2]:
# Download trained LoRA
snapshot_download(
    repo_id=hf_lora_id, 
    local_dir=lora_dir, 
    # ignore_patterns='checkpoint-*/*',
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

'/content/L3.1-8B-wikipedia-en-LoRA-v20250305134947'

# Model

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Meta-Llama-3.1-8B',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print(model)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRota

In [7]:
config = LoraConfig.from_pretrained(lora_dir)
pprint(config.__dict__)

{'_custom_modules': None,
 'alpha_pattern': {},
 'auto_mapping': None,
 'base_model_name_or_path': 'unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit',
 'bias': 'none',
 'eva_config': None,
 'exclude_modules': None,
 'fan_in_fan_out': False,
 'inference_mode': True,
 'init_lora_weights': True,
 'layer_replication': None,
 'layers_pattern': None,
 'layers_to_transform': None,
 'loftq_config': {},
 'lora_alpha': 16,
 'lora_bias': False,
 'lora_dropout': 0,
 'megatron_config': None,
 'megatron_core': 'megatron.core',
 'modules_to_save': None,
 'peft_type': <PeftType.LORA: 'LORA'>,
 'r': 8,
 'rank_pattern': {},
 'revision': None,
 'runtime_config': LoraRuntimeConfig(ephemeral_gpu_offload=False),
 'target_modules': {'down_proj',
                    'gate_proj',
                    'k_proj',
                    'o_proj',
                    'q_proj',
                    'up_proj',
                    'v_proj'},
 'task_type': 'CAUSAL_LM',
 'use_dora': False,
 'use_rslora': False}


In [8]:
# class LoraLayer(torch.nn.Module):
#     def __init__(self, base_layer, rank, alpha):
#         super().__init__()
#         self.base_layer = base_layer
#         self.rank = rank
#         self.alpha = alpha

#         in_features = getattr(base_layer, 'in_features', None)
#         out_features = getattr(base_layer, 'out_features', None)

#         if in_features is None or out_features is None:
#             raise ValueError(f"Cannot determine in_features or out_features from {base_layer}")
        
#         self.lora_A = torch.nn.Parameter(torch.randn(rank, in_features) * 0.01)
#         self.lora_B = torch.nn.Parameter(torch.randn(out_features, rank) * 0.01)
        
#         # Scaling factor
#         self.scaling = alpha / rank

#     def forward(self, x):
#         return self.base_layer(x) + (self.lora_B @ (self.lora_A @ x)) * self.scaling

#     def load_weights(self, state_dict, prefix):
#         self.lora_A.data = state_dict[f'{prefix}.lora_A.weight']
#         self.lora_B.data = state_dict[f'{prefix}.lora_B.weight']

class NeroLayer(torch.nn.Module):
    # TODO: Implement bias parameters
    def __init__(self, base_layer, rank, alpha):
        super().__init__()
        self.base_layer = base_layer
        self.alpha = alpha

        in_features = getattr(base_layer, 'in_features', None)
        out_features = getattr(base_layer, 'out_features', None)

        if in_features is None or out_features is None:
            raise ValueError(f"Cannot determine in_features or out_features from {base_layer}")
        
        # LoRA decomposition: A (down-projection) and B (up-projection)
        self.lora_A = torch.nn.Linear(in_features, rank, bias=False)  # Projects down
        self.lora_B = torch.nn.Linear(rank, out_features, bias=False) # Projects up

        # Initialize LoRA's A and B (LoRA standard: A ~ N(0, 1/rank), B ~ 0)
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        torch.nn.init.normal_(self.lora_A.weight, mean=0.0, std=std_dev)
        torch.nn.init.zeros_(self.lora_B.weight)

        # LoRA decomposition for Nero
        self.nero_A = torch.nn.Linear(out_features, rank, bias=False)
        self.nero_B = torch.nn.Linear(rank, out_features, bias=False)

        # Initialize Nero's A and B
        torch.nn.init.normal_(self.nero_A.weight, mean=0.0, std=std_dev)
        torch.nn.init.zeros_(self.nero_B.weight)
        
    def forward(self, x):
        # Base layer
        base_out = self.base_layer(x); print(f'{base_out.shape=}')

        # LoRA
        lora_A_out = self.lora_A(x); print(f'{lora_A_out.shape=}')
        lora_B_out = self.lora_B(lora_A_out); print(f'{lora_B_out.shape=}')
        lora_out = self.alpha * lora_B_out; print(f'{lora_out.shape=}')

        # Nero
        nero_A_out = self.nero_A(lora_out); print(f'{nero_A_out.shape=}')
        nero_B_out = self.nero_B(nero_A_out); print(f'{nero_B_out.shape=}')
        nero_out = self.alpha * nero_B_out; print(f'{nero_out.shape=}')
        
        return base_out + nero_out

    def load_lora_weights(self, state_dict, prefix):
        self.lora_A.weight.data = state_dict[f'{prefix}.lora_A.weight']
        self.lora_B.weight.data = state_dict[f'{prefix}.lora_B.weight']
    
class NeroModel(torch.nn.Module):
    def __init__(self, base_model: torch.nn.Module, config: LoraConfig):
        super().__init__()
        self.base_model = base_model
        self.nero_layers = torch.nn.ModuleDict() # Use _ prefix to hide the modules when printing the model

        # Wrap target layers with LoRA layers
        for module_name, module in self.base_model.named_modules():
            # Skip module if it is already a LoRA layer
            if isinstance(module, NeroLayer):
                # Store LoRA layers for weights loading
                module_name = module_name.rsplit('model.', 1)[-1]
                module_name = module_name.replace('.', '__DOT__')
                self.nero_layers[module_name] = module
                # print(f"Skipping {module_name} as it is already a LoRA layer")
                continue

            if any(module_name.endswith(target_module) for target_module in config.target_modules) and isinstance(module, torch.nn.Linear):    
                # Wrap target layer with LoRA layer
                parent_module, child_name = self._get_parent_module(module_name)
                lora_layer = NeroLayer(module, config.r, config.lora_alpha)
                setattr(parent_module, child_name, lora_layer)

                # Store LoRA layers for weights loading
                module_name = module_name.rsplit('model.', 1)[-1]
                module_name = module_name.replace('.', '__DOT__')
                self.nero_layers[module_name] = lora_layer
    
    def _get_parent_module(self, module_name):
        parts = module_name.split('.')
        parent_module = self.base_model
        for part in parts[:-1]:
            parent_module = getattr(parent_module, part)
        return parent_module, parts[-1]

    def forward(self, input_ids, attention_mask=None):
        return self.base_model(input_ids, attention_mask=attention_mask)

    def load_lora_weights(self, lora_path):
        state_dict = load_file(lora_path)
        prefix = list(state_dict.keys())[0].rsplit('model.', 1)[0] + 'model.'
        for nero_layer_name, nero_layer in self.nero_layers.items():
            nero_layer_name = nero_layer_name.replace('__DOT__', '.')
            nero_layer_name = prefix + nero_layer_name
            if f'{nero_layer_name}.lora_A.weight' in state_dict and f'{nero_layer_name}.lora_B.weight' in state_dict:
                nero_layer.load_lora_weights(state_dict, nero_layer_name)
                # print(f"Loaded LoRA weights for {nero_layer_name}")
        print("LoRA weights loaded successfully!")

nero_model = NeroModel(model, config)
print(nero_model)

NeroModel(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
      (layers): ModuleList(
        (0): LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): NeroLayer(
              (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (lora_A): Linear(in_features=4096, out_features=8, bias=False)
              (lora_B): Linear(in_features=8, out_features=4096, bias=False)
              (nero_A): Linear(in_features=4096, out_features=8, bias=False)
              (nero_B): Linear(in_features=8, out_features=4096, bias=False)
            )
            (k_proj): NeroLayer(
              (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (lora_A): Linear(in_features=4096, out_features=8, bias=False)
              (lora_B): Linear(in_features=8, out_features=1024, bias=False)
              (nero_A): Linear(in_features=1024

In [9]:
@torch.no_grad()
def check_lora_parameters(model):
    print("Check LoRA parameters:")
    for module_name, module in model.named_parameters():
        if 'lora' in module_name:
            print("Name:", module_name)
            print("Mean:", module.mean().item())
            break

check_lora_parameters(nero_model)

Check LoRA parameters:
Name: base_model.model.layers.0.self_attn.q_proj.lora_A.weight
Mean: 0.0003200761857442558


In [10]:
lora_path = os.path.join(lora_dir, 'adapter_model.safetensors')
nero_model.load_lora_weights(lora_path)

LoRA weights loaded successfully!


In [12]:
check_lora_parameters(nero_model)

Check LoRA parameters:
Name: base_model.model.layers.0.self_attn.q_proj.lora_A.weight
Mean: 4.938062556902878e-05
